In [15]:
import pandas as pd
import numpy as np 
import re
import string
from nltk.corpus import stopwords

In [16]:
df = pd.read_csv("C:/Users/Admin/NLP_Project/Movies_review_dataset/IMDB Dataset.csv")
df = pd.DataFrame(df.iloc[:2000])
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
1995,"Feeling Minnesota, directed by Steven Baigelma...",negative
1996,THE CELL (2000) Rating: 8/10<br /><br />The Ce...,positive
1997,"This movie, despite its list of B, C, and D li...",negative
1998,I loved this movie! It was all I could do not ...,positive


In [17]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [18]:
def remove_htmltag(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(remove_htmltag).str.lower()

In [19]:
print(df)

                                                 review sentiment
0     one of the other reviewers has mentioned that ...  positive
1     a wonderful little production. the filming tec...  positive
2     i thought this was a wonderful way to spend ti...  positive
3     basically there's a family where a little boy ...  negative
4     petter mattei's "love in the time of money" is...  positive
...                                                 ...       ...
1995  feeling minnesota, directed by steven baigelma...  negative
1996  the cell (2000) rating: 8/10the cell, like ant...  positive
1997  this movie, despite its list of b, c, and d li...  negative
1998  i loved this movie! it was all i could do not ...  positive
1999  this was the worst movie i have ever seen bill...  negative

[2000 rows x 2 columns]


In [20]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(remove_url)

In [21]:
def remove_punctuation(text):
    return text.translate(str.maketrans('','', string.punctuation))

df['review'] = df['review'].apply(remove_punctuation)
print(df)

                                                 review sentiment
0     one of the other reviewers has mentioned that ...  positive
1     a wonderful little production the filming tech...  positive
2     i thought this was a wonderful way to spend ti...  positive
3     basically theres a family where a little boy j...  negative
4     petter matteis love in the time of money is a ...  positive
...                                                 ...       ...
1995  feeling minnesota directed by steven baigelman...  negative
1996  the cell 2000 rating 810the cell like antz mus...  positive
1997  this movie despite its list of b c and d list ...  negative
1998  i loved this movie it was all i could do not t...  positive
1999  this was the worst movie i have ever seen bill...  negative

[2000 rows x 2 columns]


In [22]:
def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append(" ")
        else:
            new_text.append(word)
    return ' '.join(new_text)

df['review'] = df['review'].apply(remove_stopwords)
print(df)

                                                 review sentiment
0     one       reviewers   mentioned     watching  ...  positive
1       wonderful little production   filming techni...  positive
2       thought       wonderful way   spend time    ...  positive
3     basically theres   family     little boy jake ...  negative
4     petter matteis love     time   money     visua...  positive
...                                                 ...       ...
1995  feeling minnesota directed   steven baigelmann...  negative
1996    cell 2000 rating 810the cell like antz must ...  positive
1997    movie despite   list   b c     list celebs  ...  negative
1998    loved   movie         could       break     ...  positive
1999        worst movie     ever seen billy zane    ...  negative

[2000 rows x 2 columns]


In [23]:
df1 = df

In [24]:
x = df.iloc[:,0:1]
y = df['sentiment']

In [25]:
x

,review
0,one reviewers mentioned watching ...
1,wonderful little production filming techni...
2,thought wonderful way spend time ...
3,basically theres family little boy jake ...
4,petter matteis love time money visua...
...,...
1995,feeling minnesota directed steven baigelmann...
1996,cell 2000 rating 810the cell like antz must ...
1997,movie despite list b c list celebs ...
1998,loved movie could break ...


In [26]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
y = encode.fit_transform(y)

In [27]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2 , random_state=1)

In [29]:
x_train.shape

(1600, 1)

In [30]:
# applying BOW

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [31]:
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [32]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [33]:
gnb.fit(x_train_bow, y_train)

GaussianNB()

In [34]:
y_pred = gnb.predict(x_test_bow)

from sklearn.metrics import accuracy_score , confusion_matrix
accuracy_score(y_test,y_pred)

0.635

In [35]:
confusion_matrix(y_test,y_pred)

array([[136,  79],
       [ 67, 118]], dtype=int64)

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(x_train_bow,y_train)
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test, y_pred)

0.7825

In [37]:
# using TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer()

In [38]:
x_train_tfidf = tfidf.fit_transform(x_train['review']).toarray()
x_test_tfidf = tfidf.transform(x_test['review']).toarray()

In [39]:
rf = RandomForestClassifier()

rf.fit(x_train_tfidf,y_train)
y_pred = rf.predict(x_test_tfidf)
accuracy_score(y_test, y_pred)

0.8075

In [40]:
# Using Word2Vec


import gensim

from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [41]:
new_df = []

for line in df1['review']:
    raw_sent = sent_tokenize(line)
    for sent in raw_sent:
        new_df.append(simple_preprocess(sent))

In [42]:
print(type(new_df))

<class 'list'>


In [43]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

In [44]:
model.build_vocab(new_df)

In [45]:
model.train(new_df, total_examples= model.corpus_count, epochs= model.epochs)

(1030339, 1157190)

In [46]:
len(model.wv.index_to_key)

14081

In [47]:
def document_vec(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [48]:
document_vec(df['review'].values[0])

array([-0.25623345,  0.31582958,  0.02111863, -0.06299181,  0.17761964,
       -0.7161524 ,  0.189604  ,  0.9588605 , -0.44022664, -0.30031297,
       -0.2391721 , -0.70334584, -0.09306017,  0.15175168,  0.17698722,
       -0.31602004,  0.11635438, -0.4221358 , -0.02070265, -0.70681393,
        0.20645651,  0.0089289 ,  0.20279655, -0.190608  ,  0.07697164,
       -0.1021476 , -0.2661547 , -0.15959716, -0.26650187,  0.15679303,
        0.36264256,  0.11589609,  0.04879799, -0.42647487, -0.17602922,
        0.53918576,  0.14370975, -0.45103002, -0.15530996, -0.8304951 ,
       -0.0138821 , -0.40869227,  0.07489559,  0.06931018,  0.30715352,
       -0.17411344, -0.55215555, -0.06090933,  0.18968022,  0.32935825,
        0.3177582 , -0.5161934 , -0.12345362, -0.08794779, -0.3339849 ,
        0.14277919,  0.27194893, -0.02293839, -0.3487873 ,  0.00611714,
        0.20371693,  0.16430256, -0.13392246,  0.04302749, -0.42256114,
        0.2676201 ,  0.15629147,  0.28424922, -0.76435804,  0.44

In [49]:
from tqdm import tqdm

In [50]:
x= []
for doc in tqdm(df['review'].values):
    x.append(document_vec(doc))

100%|██████████| 2000/2000 [00:13<00:00, 143.75it/s]


In [51]:
x = np.array(x)

In [52]:
x[0]

array([-0.25623345,  0.31582958,  0.02111863, -0.06299181,  0.17761964,
       -0.7161524 ,  0.189604  ,  0.9588605 , -0.44022664, -0.30031297,
       -0.2391721 , -0.70334584, -0.09306017,  0.15175168,  0.17698722,
       -0.31602004,  0.11635438, -0.4221358 , -0.02070265, -0.70681393,
        0.20645651,  0.0089289 ,  0.20279655, -0.190608  ,  0.07697164,
       -0.1021476 , -0.2661547 , -0.15959716, -0.26650187,  0.15679303,
        0.36264256,  0.11589609,  0.04879799, -0.42647487, -0.17602922,
        0.53918576,  0.14370975, -0.45103002, -0.15530996, -0.8304951 ,
       -0.0138821 , -0.40869227,  0.07489559,  0.06931018,  0.30715352,
       -0.17411344, -0.55215555, -0.06090933,  0.18968022,  0.32935825,
        0.3177582 , -0.5161934 , -0.12345362, -0.08794779, -0.3339849 ,
        0.14277919,  0.27194893, -0.02293839, -0.3487873 ,  0.00611714,
        0.20371693,  0.16430256, -0.13392246,  0.04302749, -0.42256114,
        0.2676201 ,  0.15629147,  0.28424922, -0.76435804,  0.44

In [53]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

In [54]:
y = encode.fit_transform(df['sentiment'])

In [55]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [56]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train , y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [58]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test, y_pred)

0.615